<a href="https://colab.research.google.com/github/NeuralClassifier/OutlierDetect/blob/main/HDBSCAN_optimalHierarchies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [1]:
!unzip /content/HDBSCAN_outlier.zip

Archive:  /content/HDBSCAN_outlier.zip
   creating: HDBSCAN_outlier/
   creating: HDBSCAN_outlier/.ipynb_checkpoints/
   creating: __MACOSX/
   creating: __MACOSX/HDBSCAN_outlier/
  inflating: __MACOSX/HDBSCAN_outlier/._.ipynb_checkpoints  
   creating: HDBSCAN_outlier/30p/
   creating: HDBSCAN_outlier/30p/.ipynb_checkpoints/
   creating: __MACOSX/HDBSCAN_outlier/30p/
  inflating: __MACOSX/HDBSCAN_outlier/30p/._.ipynb_checkpoints  
   creating: HDBSCAN_outlier/30p/1/
  inflating: HDBSCAN_outlier/30p/1/OS_EachPt_mpts_o1d100r30_hdbLabls.xlsx  
   creating: __MACOSX/HDBSCAN_outlier/30p/1/
  inflating: __MACOSX/HDBSCAN_outlier/30p/1/._OS_EachPt_mpts_o1d100r30_hdbLabls.xlsx  
  inflating: HDBSCAN_outlier/30p/1/OS_EachPt_mpts_o1d10r30_hdbLabls.xlsx  
  inflating: __MACOSX/HDBSCAN_outlier/30p/1/._OS_EachPt_mpts_o1d10r30_hdbLabls.xlsx  
  inflating: HDBSCAN_outlier/30p/1/OS_EachPt_mpts_o1d20r30_hdbLabls.xlsx  
  inflating: __MACOSX/HDBSCAN_outlier/30p/1/._OS_EachPt_mpts_o1d20r30_hdbLabls.xlsx 

In [2]:
!unzip /content/reduced_synthetic_data_local_global.zip

Archive:  /content/reduced_synthetic_data_local_global.zip
   creating: reduced_synthetic_data_local_global/
  inflating: reduced_synthetic_data_local_global/.DS_Store  
   creating: __MACOSX/reduced_synthetic_data_local_global/
  inflating: __MACOSX/reduced_synthetic_data_local_global/._.DS_Store  
   creating: reduced_synthetic_data_local_global/30p/
   creating: reduced_synthetic_data_local_global/30p/1/
  inflating: reduced_synthetic_data_local_global/30p/1/.DS_Store  
   creating: __MACOSX/reduced_synthetic_data_local_global/30p/
   creating: __MACOSX/reduced_synthetic_data_local_global/30p/1/
  inflating: __MACOSX/reduced_synthetic_data_local_global/30p/1/._.DS_Store  
  inflating: reduced_synthetic_data_local_global/30p/1/30p_m1_dim10.xlsx  
  inflating: __MACOSX/reduced_synthetic_data_local_global/30p/1/._30p_m1_dim10.xlsx  
  inflating: reduced_synthetic_data_local_global/30p/1/30p_m1_dim100.xlsx  
  inflating: __MACOSX/reduced_synthetic_data_local_global/30p/1/._30p_m1_dim100

In [8]:
hdb_backbone_path = '/content/HDBSCAN_outlier/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]

def perf_measure(y_actual, y_hat):
  TP = 0
  FP = 0
  TN = 0
  FN = 0

  for i in range(len(y_hat)):

    if y_actual[i]==y_hat[i]==1:
      TP += 1
    if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
      FP += 1
    if y_actual[i]==y_hat[i]==0:
      TN += 1
    if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
      FN += 1

  return TP, FP, TN, FN


def LOF_tp_fp_fn_tn(d1,d2,r,o,d):
  
  mpts_list = list(d1['mpts'].unique())
  mpts_tr = []
  reduc = []
  ovr = []
  dm = []
  tp = []
  fp = []
  tn = []
  fn = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  y_actual = np.array(d2.iloc[:,-1])
  for mpts in mpts_list:
    #clear_output(wait=True)
    d1_temp = d1[d1['mpts']==mpts]
    #mpts_list=list(d1_temp['mpts'].unique())
    mpts_tr.append(mpts)
    reduc.append(r)
    ovr.append(o)
    dm.append(d)
    data = []
    #data_idx = list(d1_temp['data index'].unique())
    #print(d1_temp)
    y_hat = np.array(d1_temp.iloc[:,-1])
    TP, FP, TN, FN =  perf_measure(y_actual,y_hat)
    tp.append(TP)
    fp.append(FP)
    tn.append(TN)
    fn.append(FN)
    print('TP: ',TP,' FP: ',FP,' TN: ',TN,' FN: ',FN)
  reduc = pd.DataFrame(reduc,columns=['Outlier Quantity'])
  ovr = pd.DataFrame(ovr,columns=['Overlap'])
  dm = pd.DataFrame(dm,columns=['Dimension'])
  mpts_tr = pd.DataFrame(mpts_tr,columns=['mpts'])
  tp = pd.DataFrame(tp,columns=['TP'])
  fp = pd.DataFrame(fp,columns=['FP'])
  tn = pd.DataFrame(tn,columns=['TN'])
  fn = pd.DataFrame(fn,columns=['FN'])
  
  precAtn, adjusted_precAtn, n_out, n_in = prec_at_n_lof(d1,d2)

  return pd.concat([reduc,ovr,dm,mpts_tr,n_out, n_in,tp,fp,tn,fn,precAtn,adjusted_precAtn],axis=1)



def KNN_tp_fp_fn_tn(d1,d2,r,o,d):
  
  mpts_list = list(d1['mpts'].unique())
  mpts_tr = []
  reduc = []
  ovr = []
  dm = []
  tp = []
  fp = []
  tn = []
  fn = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  y_actual = np.array(d2.iloc[:,-2])
  for mpts in mpts_list:
    #clear_output(wait=True)
    d1_temp = d1[d1['mpts']==mpts]
    #mpts_list=list(d1_temp['mpts'].unique())
    mpts_tr.append(mpts)
    reduc.append(r)
    ovr.append(o)
    dm.append(d)
    data = []
    #data_idx = list(d1_temp['data index'].unique())
    #print(d1_temp)
    y_hat = np.array(d1_temp.iloc[:,-1])
    TP, FP, TN, FN =  perf_measure(y_actual,y_hat)
    tp.append(TP)
    fp.append(FP)
    tn.append(TN)
    fn.append(FN)
    print('TP: ',TP,' FP: ',FP,' TN: ',TN,' FN: ',FN)
  reduc = pd.DataFrame(reduc,columns=['Outlier Quantity'])
  ovr = pd.DataFrame(ovr,columns=['Overlap'])
  dm = pd.DataFrame(dm,columns=['Dimension'])
  mpts_tr = pd.DataFrame(mpts_tr,columns=['mpts'])
  tp = pd.DataFrame(tp,columns=['TP'])
  fp = pd.DataFrame(fp,columns=['FP'])
  tn = pd.DataFrame(tn,columns=['TN'])
  fn = pd.DataFrame(fn,columns=['FN'])
  
  precAtn, adjusted_precAtn, n_out, n_in = prec_at_n_lof(d1,d2)

  return pd.concat([reduc,ovr,dm,mpts_tr,n_out, n_in,tp,fp,tn,fn,precAtn,adjusted_precAtn],axis=1)




# n is the total number of outliers
def prec_at_n_lof(d1,d2):
  out_idx = []
  mpts_tr = []
  prec_at_n = []
  ap_lst = []
  d2.sort_values(by=['data index'], ascending=True,inplace=True)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      out_idx.append(d2.iloc[i,0])
  #print(out_idx)

  o = 0
  i = 0
  n = len(d2)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      o += 1
    else:
      i += 1

  n_out = []
  n_in = []
  mpts_list = list(d1['mpts'].unique())
  for mpts in mpts_list:
    mpts_tr.append(mpts)
    num_cnt =0
    d1_temp = d1[d1['mpts']==mpts]
    for i in range(len(d1_temp)):
      if d1_temp.iloc[i,-1] == 1:
        if d1_temp.iloc[i,1] in out_idx:
          num_cnt+=1
    p_at_n = float(num_cnt/len(out_idx))
    if o>n:
      npm = 1
    else:
      npm = float(o/n)
    ap = float((p_at_n-npm)/(1-npm))
    prec_at_n.append(p_at_n)
    ap_lst.append(ap)
    n_out.append(o)
    n_in.append(i)
  #print(d3)
  prec_at_n = pd.DataFrame(prec_at_n,columns=['P@n'])
  ap_lst = pd.DataFrame(ap_lst,columns=['Adjusted P@n'])
  n_out = pd.DataFrame(n_out,columns=['N_out'])
  n_in = pd.DataFrame(n_in,columns=['N_in'])
  #print(prec_at_n)
  #df_res = pd.concat([d3,prec_at_n],axis = 1)
  return prec_at_n, ap_lst, n_out, n_in

def ap_lof(d2,d3):
  o = 0
  n = len(d2)
  for i in range(len(d2)):
    if d2.iloc[i,-1] == 1:
      o += 1
  ap_lst = []
  mpts_lst = list(d3['mpts'].unique())
  for i in mpts_lst:
    ap_lst = []
    d3_temp = d3[d3['mpts']==i] 
    for j in range(len(d3_temp)):
      p_at_n = d3_temp.iloc[j,-1]
      ap = float((p_at_n-(o/n))/(1-(o/n)))
      ap_lst.append(ap)
    

In [ ]:
#LOF

hdb_backbone_path = '/content/HDBSCAN_outlier/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]

for r in redc:
  redc_path = hdb_backbone_path + str(r)+'p/'
  for o in over:
    over_path = redc_path + str(o)+'/'
    for d in dim:
      clear_output(wait=True)
      final_path_hdb = over_path+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'_hdbLabls.xlsx'
      print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d)
      df_hdb = pd.read_excel(final_path_hdb)
      df_hdb.drop(['Unnamed: 0'],axis=1,inplace=True)

      path_real = '/content/reduced_synthetic_data_local_global/'+str(r)+'p/'+str(o)+'/combined_global_local/'+str(r)+'p_m'+str(o)+'_dim'+str(d)+'_combined.xlsx'
      df_real = pd.read_excel(path_real)
      df_real.drop(['Unnamed: 0'],axis=1,inplace=True)
      if r == 30 and o == 1 and d == 2:
        df_res = LOF_tp_fp_fn_tn(df_hdb,df_real,r,o,d)
      else:
        df_res = pd.concat([df_res,LOF_tp_fp_fn_tn(df_hdb,df_real,r,o,d)],axis=0)

In [ ]:
df_res.reset_index(drop=True, inplace=True)
df_res.to_excel('/content/performance_rodm_LOF.xlsx')
df_res

In [4]:
#KNN Outlier

hdb_backbone_path = '/content/HDBSCAN_outlier/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]

for r in redc:
  redc_path = hdb_backbone_path + str(r)+'p/'
  for o in over:
    over_path = redc_path + str(o)+'/'
    for d in dim:
      clear_output(wait=True)
      final_path_hdb = over_path+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'_hdbLabls.xlsx'
      print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d)
      df_hdb = pd.read_excel(final_path_hdb)
      df_hdb.drop(['Unnamed: 0'],axis=1,inplace=True)

      path_real = '/content/reduced_synthetic_data_local_global/'+str(r)+'p/'+str(o)+'/combined_global_local/'+str(r)+'p_m'+str(o)+'_dim'+str(d)+'_combined.xlsx'
      df_real = pd.read_excel(path_real)
      df_real.drop(['Unnamed: 0'],axis=1,inplace=True)
      if r == 30 and o == 1 and d == 2:
        df_res = KNN_tp_fp_fn_tn(df_hdb,df_real,r,o,d)
      else:
        df_res = pd.concat([df_res,LOF_tp_fp_fn_tn(df_hdb,df_real,r,o,d)],axis=0)

NameError: ignored

In [ ]:
df_res.reset_index(drop=True, inplace=True)
df_res.to_excel('/content/performance_rodm_KNN.xlsx')
df_res

,Outlier Quantity,Overlap,Dimension,mpts,N_out,N_in,TP,FP,TN,FN,P@n,Adjusted P@n
0,30,1,2,2,7,339,1,27,306,6,0.142857,0.124839
1,30,1,2,3,7,339,5,23,310,2,0.714286,0.708280
2,30,1,2,4,7,339,6,22,311,1,0.857143,0.854140
3,30,1,2,5,7,339,7,21,312,0,1.000000,1.000000
4,30,1,2,6,7,339,7,21,312,0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
26683,90,4,100,215,25,219,3,2,193,22,0.120000,0.007179
26684,90,4,100,216,25,219,3,1,194,22,0.120000,0.007179
26685,90,4,100,217,25,219,2,1,194,23,0.080000,-0.037949
26686,90,4,100,218,25,219,1,1,194,24,0.040000,-0.083077


In [ ]:
df_hdb

,mpts,data index,outlier score,HDBSCAN_outliers
0,2,0,0.000000,0
1,2,1,0.240688,0
2,2,2,0.000000,0
3,2,3,0.031608,0
4,2,4,0.275930,0
...,...,...,...,...
114915,339,335,0.000000,0
114916,339,336,0.000000,0
114917,339,337,0.000000,0
114918,339,338,0.000000,0


In [ ]:
df_real.sort_values(by=['data index'], ascending=True,inplace=True)
df_real

,data index,A1,A2,Y,normalized_LOF,knn_outlier,LOF_outlier
23,0,2.764052,1.400157,0,0.427347,1,0
15,1,1.978738,3.240893,0,0.492060,1,0
24,2,2.867558,0.022722,0,0.425059,1,0
303,3,1.950088,0.848643,0,0.032013,0,0
278,4,0.896781,1.410599,0,0.038716,0,0
...,...,...,...,...,...,...,...
83,335,-1.259781,0.058599,1,0.179400,0,0
116,336,-0.657871,0.049916,1,0.122259,0,0
155,337,-0.797401,1.412470,1,0.086318,0,0
122,338,0.174586,-0.977099,1,0.111748,0,0


In [ ]:
prec_at_n_lof(df_hdb,df_real,df_res)

     Outlier Quantity  Overlap  Dimension  mpts  TP  FP   TN  FN
0                  30        1          2     2   1  27  306   6
1                  30        1          2     3   5  23  310   2
2                  30        1          2     4   6  22  311   1
3                  30        1          2     5   7  21  312   0
4                  30        1          2     6   7  21  312   0
..                ...      ...        ...   ...  ..  ..  ...  ..
213                90        4        100   215   3   2  193  22
214                90        4        100   216   3   1  194  22
215                90        4        100   217   2   1  194  23
216                90        4        100   218   1   1  194  24
217                90        4        100   219   0   1  194  25

[26688 rows x 8 columns]
     PrecAtN
0       0.44
1       0.36
2       0.20
3       0.20
4       0.20
..       ...
213     0.12
214     0.12
215     0.08
216     0.04
217     0.00

[218 rows x 1 columns]


ValueError: ignored